# Model Deployment and Making Inferences

In this notebook we will be using our trained model to make local inferences. We will get predictions from our validation/test set and evaluate the performance of our model.

- load our trained mode
- configuring and deploying the model endpoint on SageMaker
- make local inferences
- predictions on validation set
- confusion matrix

In [1]:
import sagemaker
import boto3
import os

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
bucket = 'not-hot-dog'

print(role)
print(region)

arn:aws:iam::003294323742:role/service-role/AmazonSageMaker-ExecutionRole-20230115T105791
us-east-2


## Loading our trained model

We will be loading the trained model that achieved the highest accuracy during hyperparemeter tuning.

In [2]:
# Definining algorithm image
from sagemaker import image_uris

container = image_uris.retrieve(
    region = region,
    framework = 'image-classification'
)

In [3]:
# Path to the best model
model_path = 's3://not-hot-dog/models/image_model/image-classification-230118-0234-001-d91e93a4/output/model.tar.gz'

model = sagemaker.model.Model(
    image_uri = container,
    model_data = model_path,
    role = role
)

## Configuring and Deploying Endpoint

In [4]:
# creating endpoint
model_endpoint_name = 'hotdog-classifier'

deployed_model = model.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    endpoint_name = model_endpoint_name
)

------------!

## Making Local Inferences

In [6]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
import base64

predictor = Predictor(model_endpoint_name)

file_name = './data/validation/hot_dog_0.jpg'

predictor.serializer = IdentitySerializer('image/jpeg')
with open(file_name, 'rb') as f:
    payload = f.read()
    
inference = predictor.predict(data = payload)
# left: probability not hotdog, right: probability hot dog
print(inference)

b'[0.4506995379924774, 0.549300491809845]'


predicted that this image is 54.93% a hot dog. This is a byte array; we need to deserialize it.

## Predictions on Validation Test

In [7]:
import glob 
import json
import numpy as np

y_true = []
y_pred = []

validation_file_path = './data/validation/*.jpg'

def make_predictions(validation_path):
    files = glob.glob(validation_path)
    for file in files:
        if 'not_hot_dog' in file:
            with open(file, 'rb') as f:
                payload = f.read()
                inference = predictor.predict(data = payload).decode('utf-8')
                result = json.loads(inference)
                predicted_class = np.argmax(result)
                y_true.append(0)
                y_pred.append(predicted_class)
        elif '/hot_dog' in file:
            with open(file, 'rb') as f:
                payload = f.read()
                inference = predictor.predict(data = payload).decode('utf-8')
                result = json.loads(inference)
                predicted_class = np.argmax(result)
                y_true.append(1)
                y_pred.append(predicted_class)
                
make_predictions(validation_file_path)
print(y_true)
print(y_pred)

[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0]
[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,

## Confusion Matrix

In [8]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

array([[67, 33],
       [38, 61]])